# Federated Learning Training

Este notebook treina modelos LSTM com Federated Learning para ambos os datasets.

## O que este notebook faz:
1. Carrega dados processados (Sleep-EDF e WESAD)
2. Treina modelos LSTM com FL para diferentes números de clientes
3. Avalia performance e custo de comunicação
4. Salva modelos e resultados
5. Cria visualizações da convergência

**Pré-requisitos**: Execute os notebooks 00, 01, 02 e 03 primeiro.


In [ ]:
# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================

print("="*70)
print("FEDERATED LEARNING TRAINING")
print("="*70)

# Import modules
from src.privacy.fl_training import train_with_fl, evaluate_fl_model, save_fl_model, get_fl_configs
from src.preprocessing.sleep_edf import load_processed_sleep_edf
from src.preprocessing.wesad import load_processed_wesad
from src.evaluation.visualization import plot_fl_convergence, plot_comparison_bars
import numpy as np
import os

# Define paths
SLEEP_DATA_PATH = '/content/drive/MyDrive/mhealth-data/processed/sleep-edf'
WESAD_DATA_PATH = '/content/drive/MyDrive/mhealth-data/processed/wesad'
MODELS_PATH = '/content/drive/MyDrive/mhealth-data/models'
RESULTS_PATH = '/content/drive/MyDrive/mhealth-data/results'

print("Loading processed data...")

# Load Sleep-EDF data
if os.path.exists(SLEEP_DATA_PATH):
    X_train_sleep, X_val_sleep, X_test_sleep, y_train_sleep, y_val_sleep, y_test_sleep, _, _, sleep_info = load_processed_sleep_edf(SLEEP_DATA_PATH)
    print(f"✅ Sleep-EDF loaded: {X_train_sleep.shape}")
else:
    print("❌ Sleep-EDF data not found. Run notebook 01 first.")
    X_train_sleep = None

# Load WESAD data
if os.path.exists(WESAD_DATA_PATH):
    X_train_wesad, X_val_wesad, X_test_wesad, y_train_wesad, y_val_wesad, y_test_wesad, _, _, wesad_info = load_processed_wesad(WESAD_DATA_PATH)
    print(f"✅ WESAD loaded: {X_train_wesad.shape}")
else:
    print("❌ WESAD data not found. Run notebook 02 first.")
    X_train_wesad = None

# Define client numbers to test
n_clients_list = [3, 5, 10]
print(f"\nClient numbers to test: {n_clients_list}")


In [ ]:
# ============================================================================
# STEP 2: Train Sleep-EDF FL Models
# ============================================================================

if X_train_sleep is not None:
    print("\n" + "="*70)
    print("TRAINING SLEEP-EDF FL MODELS")
    print("="*70)
    
    sleep_fl_results = {}
    
    for n_clients in n_clients_list:
        print(f"\n--- Training with {n_clients} clients ---")
        
        # Get FL configuration
        fl_configs = get_fl_configs([n_clients])
        config = fl_configs[0]
        config.update({
            'dataset': 'sleep_edf',
            'model_type': 'fl',
            'privacy_technique': 'FL',
            'privacy_parameter': f'{n_clients} clients'
        })
        
        # Train FL model
        model_fl, history_fl, fl_info = train_with_fl(
            X_train_sleep, y_train_sleep,
            X_val_sleep, y_val_sleep,
            config
        )
        
        # Evaluate model
        results_fl = evaluate_fl_model(model_fl, X_test_sleep, y_test_sleep, config['window_size'])
        results_fl.update({
            'dataset': 'sleep_edf',
            'model_type': 'fl',
            'privacy_technique': 'FL',
            'privacy_parameter': f'{n_clients} clients',
            'n_clients': n_clients,
            'communication_cost': fl_info['communication_cost']
        })
        
        # Save model and results
        model_name = f'sleep_edf_fl_{n_clients}_clients'
        save_fl_model(
            model_fl, history_fl, results_fl, fl_info,
            MODELS_PATH, model_name
        )
        
        # Store results
        sleep_fl_results[f'Sleep-EDF FL ({n_clients} clients)'] = results_fl
        
        print(f"✅ Sleep-EDF FL model ({n_clients} clients) trained successfully!")
        print(f"Test Accuracy: {results_fl['accuracy']:.4f}")
        print(f"Communication Cost: {fl_info['communication_cost']} rounds")
    
    print(f"\n✅ All Sleep-EDF FL models trained!")
    
else:
    print("❌ Skipping Sleep-EDF FL training - data not available")
    sleep_fl_results = {}


In [ ]:
# ============================================================================
# STEP 3: Train WESAD FL Models
# ============================================================================

if X_train_wesad is not None:
    print("\n" + "="*70)
    print("TRAINING WESAD FL MODELS")
    print("="*70)
    
    wesad_fl_results = {}
    
    for n_clients in n_clients_list:
        print(f"\n--- Training with {n_clients} clients ---")
        
        # Get FL configuration
        fl_configs = get_fl_configs([n_clients])
        config = fl_configs[0]
        config.update({
            'dataset': 'wesad',
            'model_type': 'fl',
            'privacy_technique': 'FL',
            'privacy_parameter': f'{n_clients} clients'
        })
        
        # Train FL model
        model_fl, history_fl, fl_info = train_with_fl(
            X_train_wesad, y_train_wesad,
            X_val_wesad, y_val_wesad,
            config
        )
        
        # Evaluate model
        results_fl = evaluate_fl_model(model_fl, X_test_wesad, y_test_wesad, config['window_size'])
        results_fl.update({
            'dataset': 'wesad',
            'model_type': 'fl',
            'privacy_technique': 'FL',
            'privacy_parameter': f'{n_clients} clients',
            'n_clients': n_clients,
            'communication_cost': fl_info['communication_cost']
        })
        
        # Save model and results
        model_name = f'wesad_fl_{n_clients}_clients'
        save_fl_model(
            model_fl, history_fl, results_fl, fl_info,
            MODELS_PATH, model_name
        )
        
        # Store results
        wesad_fl_results[f'WESAD FL ({n_clients} clients)'] = results_fl
        
        print(f"✅ WESAD FL model ({n_clients} clients) trained successfully!")
        print(f"Test Accuracy: {results_fl['accuracy']:.4f}")
        print(f"Communication Cost: {fl_info['communication_cost']} rounds")
    
    print(f"\n✅ All WESAD FL models trained!")
    
else:
    print("❌ Skipping WESAD FL training - data not available")
    wesad_fl_results = {}


In [ ]:
# ============================================================================
# STEP 4: Create FL Visualizations
# ============================================================================

print("\n" + "="*70)
print("CREATING FL VISUALIZATIONS")
print("="*70)

# Combine all FL results
all_fl_results = {}
all_fl_results.update(sleep_fl_results)
all_fl_results.update(wesad_fl_results)

if all_fl_results:
    # Create comparison bars for Sleep-EDF
    if sleep_fl_results:
        plot_comparison_bars(
            sleep_fl_results,
            metrics=['accuracy', 'f1_score'],
            save_path=f'{RESULTS_PATH}/fl_comparison_sleep_edf.png',
            title='Sleep-EDF: FL Performance Comparison'
        )
    
    # Create comparison bars for WESAD
    if wesad_fl_results:
        plot_comparison_bars(
            wesad_fl_results,
            metrics=['accuracy', 'f1_score'],
            save_path=f'{RESULTS_PATH}/fl_comparison_wesad.png',
            title='WESAD: FL Performance Comparison'
        )
    
    # Save all FL results
    from src.evaluation.metrics import save_evaluation_results
    save_evaluation_results(all_fl_results, RESULTS_PATH, 'fl_results.json')
    
    print("✅ FL visualizations created and results saved!")
    
else:
    print("❌ No FL results to visualize")


In [ ]:
# ============================================================================
# STEP 5: Summary and Next Steps
# ============================================================================

print("\n" + "="*70)
print("FL TRAINING COMPLETE!")
print("="*70)

# Print summary
if sleep_fl_results:
    print("\nSleep-EDF FL Results:")
    for model_name, results in sleep_fl_results.items():
        print(f"  {model_name}: Accuracy={results['accuracy']:.4f}, F1={results['f1_score']:.4f}, Cost={results['communication_cost']}")

if wesad_fl_results:
    print("\nWESAD FL Results:")
    for model_name, results in wesad_fl_results.items():
        print(f"  {model_name}: Accuracy={results['accuracy']:.4f}, F1={results['f1_score']:.4f}, Cost={results['communication_cost']}")

print(f"\nModels saved to: {MODELS_PATH}")
print(f"Results saved to: {RESULTS_PATH}")

print("\nNext steps:")
print("1. Run notebook 06_analysis.ipynb for comprehensive analysis")

print("\n✅ FL models are ready for final analysis and comparison!")
